In [1]:
import pandas as pd
df = pd.read_csv('file_with_gis_community_boards.csv', nrows=148701)  # Loads rows

In [3]:
df.isnull().sum()

unique_key                            0
created_date                          0
closed_date                          34
complaint_type                        0
descriptor                            0
incident_zip                       5663
incident_address                  53155
street_name                       53160
cross_street_1                    51138
cross_street_2                    51170
intersection_street_1             98004
intersection_street_2             98006
address_type                          0
city                               5615
resolution_description                0
resolution_action_updated_date        0
community_board                    3103
borough                            3103
latitude                            451
longitude                           451
location                            450
community_board_GIS                 652
dtype: int64

In [5]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [unique_key, created_date, closed_date, complaint_type, descriptor, incident_zip, incident_address, street_name, cross_street_1, cross_street_2, intersection_street_1, intersection_street_2, address_type, city, resolution_description, resolution_action_updated_date, community_board, borough, latitude, longitude, location, community_board_GIS]
Index: []

[0 rows x 22 columns]


In [7]:
print(df.describe())

         unique_key   incident_zip       latitude      longitude  \
count  1.487010e+05  143038.000000  148250.000000  148250.000000   
mean   5.428972e+07   10904.544967      40.709651     -73.904971   
std    5.152954e+06     526.998689       0.259159       1.874033   
min    4.527865e+07      83.000000      18.684809     -75.255729   
25%    5.038539e+07   10451.000000      40.648768     -73.982200   
50%    5.407570e+07   11211.000000      40.709739     -73.920296   
75%    5.839216e+07   11363.000000      40.763035     -73.849107   
max    6.359548e+07   11697.000000      43.299428     100.800005   

       community_board_GIS  
count        148049.000000  
mean            325.326932  
std             119.818484  
min             101.000000  
25%             211.000000  
50%             316.000000  
75%             410.000000  
max             595.000000  


In [9]:
# Remove rows where 'community_board_GIS' is null
df_cleaned = df.dropna(subset=['community_board_GIS','closed_date'])
df_cleaned.isnull().sum()

unique_key                            0
created_date                          0
closed_date                           0
complaint_type                        0
descriptor                            0
incident_zip                       5085
incident_address                  52551
street_name                       52555
cross_street_1                    50626
cross_street_2                    50657
intersection_street_1             97381
intersection_street_2             97381
address_type                          0
city                               5039
resolution_description                0
resolution_action_updated_date        0
community_board                    2592
borough                            2592
latitude                              0
longitude                             0
location                              0
community_board_GIS                   0
dtype: int64

In [11]:
# Ensure df_cleaned is a full copy, not a slice
df_cleaned = df_cleaned.copy()
def fill_community_board(row):
    community_board = row['community_board']
    community_board_GIS = str(row['community_board_GIS'])  # Convert to string for comparison

    # Check if the community_board is null or blank
    if pd.isnull(community_board) or community_board.strip() == '' or 'Unspecified' in community_board:
        # Replace with the value derived from community_board_GIS
        if community_board_GIS.startswith('1'):
            return f"{int(float(community_board_GIS[1:])):02d} MANHATTAN"
        elif community_board_GIS.startswith('2'):
            return f"{int(float(community_board_GIS[1:])):02d} BRONX"
        elif community_board_GIS.startswith('3'):
            return f"{int(float(community_board_GIS[1:])):02d} BROOKLYN"
        elif community_board_GIS.startswith('4'):
            return f"{int(float(community_board_GIS[1:])):02d} QUEENS"
        elif community_board_GIS.startswith('5'):
            return f"{int(float(community_board_GIS[1:])):02d} STATEN ISLAND"
    return community_board  # Return the original value if valid

# Apply the function to fill in missing/incorrect values
df_cleaned['community_board'] = df_cleaned.apply(fill_community_board, axis=1)
# Save the updated DataFrame to a new CSV file
df_cleaned.to_csv('updated_community_board_with_GIS.csv', index=False)

In [13]:
# Function to remove .0 from any column value
# def remove_dot_zero(value):
#     if isinstance(value, str) and '.0' in value:
#         return value.replace('.0', '')  # Remove the '.0' from the string
#     elif isinstance(value, float) and value.is_integer():
#         return str(int(value))  # Convert float to integer as a string
#     return value  # Return the original value if no change is needed

# Apply the function to both columns
# df_cleaned['community_board'] = df_cleaned['community_board'].apply(remove_dot_zero)
# df_cleaned['community_board_GIS'] = df_cleaned['community_board_GIS'].apply(remove_dot_zero)

# Verify changes
# print("Updated columns:")
# print(df_cleaned[['community_board', 'community_board_GIS']].head())

# Save the updated DataFrame to a new CSV file
# df_cleaned.to_csv('community_board_and_GIS_no_dot_zero.csv', index=False)


In [15]:
# Check if any '.0' exists in the community_board column
# dot_zero_values = df_cleaned['community_board_GIS'].str.contains(r'\.0', na=False)

# # Count the number of rows with '.0' in community_board
# dot_zero_count = dot_zero_values.sum()

# # Print the result
# if dot_zero_count > 0:
#     print(f"There are {dot_zero_count} rows with '.0' in the community_board column.")
#     print(df_cleaned[dot_zero_values])  # Display rows with '.0' values
# else:
#     print("No '.0' values found in the community_board column.")


In [23]:
# Function to check mismatches
def find_mismatch(row):
    community_board = row['community_board']
    community_board_GIS = str(row['community_board_GIS'])  # Convert to string for comparison

    # Handle missing or null values
    if pd.isnull(community_board) or pd.isnull(community_board_GIS):
        return 0  # Treat missing values as no mismatch

    # Extract borough and board number
    board_number, borough = community_board.split(maxsplit=1)
    if borough == 'MANHATTAN' and not community_board_GIS.startswith('1' + board_number):
        return 1
    elif borough == 'BRONX' and not community_board_GIS.startswith('2' + board_number):
        return 1
    elif borough == 'BROOKLYN' and not community_board_GIS.startswith('3' + board_number):
        return 1
    elif borough == 'QUEENS' and not community_board_GIS.startswith('4' + board_number):
        return 1
    elif borough == 'STATEN ISLAND' and not community_board_GIS.startswith('5' + board_number):
        return 1
    return 0

# Apply the function to check mismatches
df_cleaned['mismatch'] = df_cleaned.apply(find_mismatch, axis=1)
mismatch_count = df_cleaned['mismatch'].sum()

# Print the number of mismatches
print(f"Number of mismatches between 'community_board' and 'community_board_GIS': {mismatch_count}")

# Show rows with mismatches
mismatches = df_cleaned[df_cleaned['mismatch'] == 1]
print("Rows with mismatches:")
print(mismatches[['community_board', 'community_board_GIS', 'location']])

# Save the updated DataFrame to a new CSV file
df_cleaned.to_csv('check_mismatch_8508.csv', index=False)


Number of mismatches between 'community_board' and 'community_board_GIS': 264
Rows with mismatches:
       community_board  community_board_GIS  \
493        16.0 QUEENS                316.0   
682        18.0 QUEENS                318.0   
856      05.0 BROOKLYN                405.0   
899        16.0 QUEENS                316.0   
915        16.0 QUEENS                316.0   
...                ...                  ...   
136746   05.0 BROOKLYN                405.0   
137991   05.0 BROOKLYN                405.0   
142773     04.0 QUEENS                304.0   
146666   05.0 BROOKLYN                405.0   
147750   05.0 BROOKLYN                405.0   

                                                 location  
493                   ATLANTIC AVENUE, QUEENS, NY 11421.0  
682                      BELT PARKWAY, QUEENS, NY 11414.0  
856     IRVING AVENUE and MOFFAT STREET, BROOKLYN, NY ...  
899                   ATLANTIC AVENUE, QUEENS, NY 11421.0  
915                   ATLANTIC AVEN

In [19]:
# Function to check mismatches and fix them if needed
def fix_mismatch(row):
    community_board = row['community_board']
    community_board_GIS = str(row['community_board_GIS'])  # Convert to string for comparison

    # Handle missing or null values
    if pd.isnull(community_board) or pd.isnull(community_board_GIS):
        return community_board  # Keep as is for missing values

    # Extract borough and board number from community_board
    try:
        board_number, borough = community_board.split(maxsplit=1)
    except ValueError:
        return community_board  # Keep as is if split fails

    # Check for mismatch and fix it
    if borough == 'MANHATTAN' and not community_board_GIS.startswith('1' + board_number):
        return f"{community_board_GIS[1:].zfill(2)} MANHATTAN"
    elif borough == 'BRONX' and not community_board_GIS.startswith('2' + board_number):
        return f"{community_board_GIS[1:].zfill(2)} BRONX"
    elif borough == 'BROOKLYN' and not community_board_GIS.startswith('3' + board_number):
        return f"{community_board_GIS[1:].zfill(2)} BROOKLYN"
    elif borough == 'QUEENS' and not community_board_GIS.startswith('4' + board_number):
        return f"{community_board_GIS[1:].zfill(2)} QUEENS"
    elif borough == 'STATEN ISLAND' and not community_board_GIS.startswith('5' + board_number):
        return f"{community_board_GIS[1:].zfill(2)} STATEN ISLAND"

    return community_board  # If no mismatch, return the original value

# Apply the function to fix mismatches
df_cleaned['community_board'] = df_cleaned.apply(fix_mismatch, axis=1)

# Function to identify remaining mismatches
def find_mismatch(row):
    community_board = row['community_board']
    community_board_GIS = str(row['community_board_GIS'])  # Convert to string for comparison

    # Handle missing or null values
    if pd.isnull(community_board) or pd.isnull(community_board_GIS):
        return 1  # Count as a mismatch if missing values

    # Extract borough and board number
    try:
        board_number, borough = community_board.split(maxsplit=1)
    except ValueError:
        return 1  # Count as a mismatch if invalid community_board format

    # Check for mismatch
    if borough == 'MANHATTAN' and not community_board_GIS.startswith('1' + board_number):
        return 1
    elif borough == 'BRONX' and not community_board_GIS.startswith('2' + board_number):
        return 1
    elif borough == 'BROOKLYN' and not community_board_GIS.startswith('3' + board_number):
        return 1
    elif borough == 'QUEENS' and not community_board_GIS.startswith('4' + board_number):
        return 1
    elif borough == 'STATEN ISLAND' and not community_board_GIS.startswith('5' + board_number):
        return 1

    return 0  # No mismatch

# Reapply mismatch check to validate the fixes
df_cleaned['mismatch'] = df_cleaned.apply(find_mismatch, axis=1)

# Count the number of remaining mismatches
mismatch_count = df_cleaned['mismatch'].sum()

# Print the number of mismatches remaining
print(f"Number of mismatches between 'community_board' and 'community_board_GIS' after fixing: {mismatch_count}")

# Show rows with remaining mismatches
remaining_mismatches = df_cleaned[df_cleaned['mismatch'] == 1]
print("Rows with remaining mismatches:")
print(remaining_mismatches[['community_board', 'community_board_GIS', 'location']])

# Save remaining mismatches to a new CSV for review
remaining_mismatches.to_csv('remaining_mismatches.csv', index=False)

# Save the updated DataFrame to a new CSV file
df_cleaned.to_csv('check_mismatch_264.csv', index=False)

Number of mismatches between 'community_board' and 'community_board_GIS' after fixing: 264
Rows with remaining mismatches:
       community_board  community_board_GIS  \
493        16.0 QUEENS                316.0   
682        18.0 QUEENS                318.0   
856      05.0 BROOKLYN                405.0   
899        16.0 QUEENS                316.0   
915        16.0 QUEENS                316.0   
...                ...                  ...   
136746   05.0 BROOKLYN                405.0   
137991   05.0 BROOKLYN                405.0   
142773     04.0 QUEENS                304.0   
146666   05.0 BROOKLYN                405.0   
147750   05.0 BROOKLYN                405.0   

                                                 location  
493                   ATLANTIC AVENUE, QUEENS, NY 11421.0  
682                      BELT PARKWAY, QUEENS, NY 11414.0  
856     IRVING AVENUE and MOFFAT STREET, BROOKLYN, NY ...  
899                   ATLANTIC AVENUE, QUEENS, NY 11421.0  
915         

In [45]:
# Remove rows where 'mismatch' equals 1
df_cleaned = df_cleaned[df_cleaned['mismatch'] != 1]

# Reset the index (optional, for clean indexing)
df_cleaned.reset_index(drop=True, inplace=True)

# Save the updated DataFrame to a new CSV file
df_cleaned.to_csv('cleaned_data_no_mismatches.csv', index=False)

# Print confirmation and a preview
print("Rows with 'mismatch' == 1 have been removed.")
print(df_cleaned.head())

Rows with 'mismatch' == 1 have been removed.
   unique_key             created_date              closed_date  \
0    63589056  2024-12-31T23:24:25.000  2025-01-01T21:55:00.000   
1    63587853  2024-12-31T23:23:19.000  2025-01-01T21:05:00.000   
2    63594198  2024-12-31T23:22:08.000  2025-01-01T19:10:00.000   
3    63589057  2024-12-31T23:09:09.000  2025-01-01T17:20:00.000   
4    63587916  2024-12-31T14:09:35.000  2025-01-01T13:30:00.000   

     complaint_type descriptor  incident_zip   incident_address  \
0  Street Condition    Pothole       11378.0                NaN   
1  Street Condition    Pothole       11378.0                NaN   
2  Street Condition    Pothole       11385.0                NaN   
3  Street Condition    Pothole       11385.0                NaN   
4  Street Condition    Pothole       11222.0  457 GRAHAM AVENUE   

     street_name  cross_street_1 cross_street_2  ...      city  \
0            NaN             NaN            NaN  ...    QUEENS   
1            NaN 

In [47]:
# Check if any rows with 'mismatch' == 1 exist
if (df_cleaned['mismatch'] == 1).any():
    print("Error: There are still rows with 'mismatch' == 1 in the DataFrame.")
else:
    print("Success: All rows with 'mismatch' == 1 have been removed.")


Success: All rows with 'mismatch' == 1 have been removed.


In [49]:
# Function to update the borough column based on community_board
def update_borough(row):
    community_board = row['community_board']
    borough = row['borough']

    # Check if community_board is valid
    if pd.isnull(community_board) or not isinstance(community_board, str):
        return borough  # Keep the existing borough if community_board is null or invalid

    # Extract the borough from community_board
    try:
        _, correct_borough = community_board.split(maxsplit=1)
        # Replace borough if it's 'Unspecified', blank, or invalid
        if pd.isnull(borough) or borough.strip() == '' or borough == 'Unspecified':
            return correct_borough
        return correct_borough  # Replace with the correct borough from community_board
    except ValueError:
        # If splitting fails, keep the existing borough
        return borough

# Apply the function to the borough column
df_cleaned['borough'] = df_cleaned.apply(update_borough, axis=1)

# Save the updated DataFrame to a new CSV file
df_cleaned.to_csv('updated_borough_from_community_board_new.csv', index=False)

# Print a preview of the updated DataFrame
print("Updated borough column:")
print(df_cleaned[['community_board', 'borough']].head())

Updated borough column:
  community_board   borough
0       02 QUEENS    QUEENS
1       02 QUEENS    QUEENS
2       05 QUEENS    QUEENS
3       05 QUEENS    QUEENS
4     01 BROOKLYN  BROOKLYN


In [51]:
df_cleaned.isnull().sum()

unique_key                            0
created_date                          0
closed_date                           0
complaint_type                        0
descriptor                            0
incident_zip                       5085
incident_address                  52508
street_name                       52512
cross_street_1                    50583
cross_street_2                    50614
intersection_street_1             97160
intersection_street_2             97160
address_type                          0
city                               5039
resolution_description                0
resolution_action_updated_date        0
community_board                       0
borough                               0
latitude                              0
longitude                             0
location                              0
community_board_GIS                   0
mismatch                              0
dtype: int64